In [2]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
import pytorch_lightning as pl

## T5 

In [3]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, inputs, attention_mask):
        self.inputs = inputs
        self.attention_mask = attention_mask

    # 학습 및 추론 과정에서 데이터를 1개씩 꺼내옴
    def __getitem__(self, idx):
        # 정답이 있다면 else문을, 없다면 if문을 수행
        return (torch.tensor(self.inputs[idx]), torch.tensor(self.attention_mask[idx]))
      

    # 입력하는 개수만큼 데이터를 사용
    def __len__(self):
        return len(self.inputs)

In [4]:
class Dataloader(pl.LightningDataModule):

    def __init__(self,
                 model_name,
                 batch_size,
                 path):
        
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.max_token_length = 64

        self.path = path
   
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def prepare_data(self) -> None: 
        """ csv 파일을 경로에 맞게 불러 옵니다. """
        self.dataframe = pd.read_csv(self.path)
        self.inputs, self.attention_mask = self.preprocessing(self.dataframe)

    def preprocessing(self, data):
        # 안쓰는 컬럼 삭제
        data = data["text"].tolist()
        tokenizer_input = self.tokenizer(data, padding=True, max_length=self.max_token_length, truncation=True)
        
        return tokenizer_input["input_ids"], tokenizer_input["attention_mask"]

    def setup(self, stage='predict'): 
        self.dataset = Dataset(self.inputs, self.attention_mask)

    def predict_dataloader(self): 
        return torch.utils.data.DataLoader(self.dataset, batch_size=self.batch_size)

In [5]:
class Model(pl.LightningModule):
    
    def __init__(self, kor2eng_model_name, eng2kor_model_name):
        
        super().__init__()
        self.save_hyperparameters()
        self.kor2eng_model = AutoModelForSeq2SeqLM.from_pretrained(kor2eng_model_name)
        self.eng2kor_model = AutoModelForSeq2SeqLM.from_pretrained(eng2kor_model_name)
        self.eng2kor_tokenizer = AutoTokenizer.from_pretrained(eng2kor_model_name)
        self.max_token_length = 64
        self.num_beams=5
        
    def forward(self,
                input_ids=None,
                attention_mask=None):
        
        # print(len(input_ids), attention_mask)
        eng_encoded_output = self.kor2eng_model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        max_length=self.max_token_length,
        num_beams=self.num_beams)
        
        new_attention_mask = eng_encoded_output.bool().long()
        
        kor_encoded_output = self.eng2kor_model.generate(
        input_ids = eng_encoded_output,
        attention_mask = new_attention_mask,
        max_length=self.max_token_length,
        num_beams=self.num_beams)
        
        return kor_encoded_output
                
    def predict_step(self, batch, batch_idx):
        x = batch
        
        kor_encoded_output = self(*x)
        return kor_encoded_output


In [6]:
model = Model("./kor2eng", "./eng2kor")
dataloader = Dataloader("./kor2eng", 32, '/opt/ml/train_spell_checked_delete_noise_all.csv')
trainer = pl.Trainer(
        gpus = 1,
        accelerator='gpu')

kor_encoded_output = trainer.predict(model=model, datamodule=dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [7]:
eng2kor_tokenizer = AutoTokenizer.from_pretrained("./eng2kor")
data = pd.read_csv('/opt/ml/train_spell_checked_delete_noise_all.csv')
result = []
for i in kor_encoded_output:
    result.extend(eng2kor_tokenizer.batch_decode(i, skip_special_tokens=True))

data["back_translation"] = result

In [8]:
data.to_csv("back-translation.csv", index=False)
data.head()

,ID,text,target,url,date,prob,new_label,back_translation
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36,[5.4213288e-03 2.6925930e-03 1.7177768e-02 9.6...,3,다음 달 2일부터 유튜브의 개발자 지원 공간을 다음 달 2일까지 운영한다.
1,ynat-v1_train_00001,어버이날 막다가 흐려 저…남부 지방 여하튼 황사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25,[0.00201493 0.00196364 0.00275715 0.9861522 0...,3,"오랜만에 생일날 비가 나고, 남부지방에 비가 오고 있어, 남부지방에 비가 오고 있습니다."
2,ynat-v1_train_00002,내년부터 국가 RD 평가 때 논문 건수는 반영 않는다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00,[0.05256948 0.01216032 0.7787355 0.01706305 0...,2,내년부터는 국수 RD평가에 서류 양이 반영될 예정이다.
3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54,[0.30010435 0.01159117 0.6572125 0.00868781 0...,2,김명자 국립과학기술원 신임 이사장이 어르신들과 젊은 과학자를 출산한다.
4,ynat-v1_train_00004,회생 긴 간 작가 김동식 걍심꼬백 뜬 새 소설집 두 권 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05,[0.00760565 0.00409387 0.00674626 0.96570086 0...,3,장기작가인 김동식이 장기작가 김동식과 함께 새로운 소설 두 권을 저서로 출간했다.
